In [0]:
import numpy as np
import pandas as pd
import pyspark.pandas as ps

In [0]:
# Import data from S3
from pyspark import SparkFiles

dbutils.fs.ls("s3://project-4-ag-2022/data.gzip")

df = spark.read.format("parquet").load("s3://project-4-ag-2022/data.gzip", header=True, inferSchema=True)

In [0]:
df.show()

In [0]:
import sys

from operator import add
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
import pyspark.sql.functions as f

In [0]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['body']))
data_df.show()

+-----------------+--------------------+--------------------+-------------------+------------+-------+--------------------+--------------------+------+
|           author|                body|      normalizedBody|          subreddit|subreddit_id|     id|             content|             summary|length|
+-----------------+--------------------+--------------------+-------------------+------------+-------+--------------------+--------------------+------+
|splagaticusxoxo97|FALSE. Evidence: ...|FALSE. Evidence: ...|            atheism|    t5_2qh2p|c6bacqq|FALSE. Evidence: ...|dont fuck with re...|   770|
|         phyzishy|Yeah, but most fo...|Yeah, but most fo...|          AskReddit|    t5_2qh1i|c6b52m8|Yeah, but most fo...|       stupid stuff.|   446|
|       Perservere|Didn't they lose ...|Didn't they lose ...|    leagueoflegends|    t5_2rfxx|c6bftvc|Didn't they lose ...|just because you'...|   569|
|      fallsuspect|You probably won'...|You probably won'...|          AskReddit|    t5_

In [0]:
data_df = data_df.filter(data_df['body'] != '')

In [0]:
data_df.show()

+-----------------+--------------------+--------------------+-------------------+------------+-------+--------------------+--------------------+------+
|           author|                body|      normalizedBody|          subreddit|subreddit_id|     id|             content|             summary|length|
+-----------------+--------------------+--------------------+-------------------+------------+-------+--------------------+--------------------+------+
|splagaticusxoxo97|FALSE. Evidence: ...|FALSE. Evidence: ...|            atheism|    t5_2qh2p|c6bacqq|FALSE. Evidence: ...|dont fuck with re...|   770|
|         phyzishy|Yeah, but most fo...|Yeah, but most fo...|          AskReddit|    t5_2qh1i|c6b52m8|Yeah, but most fo...|       stupid stuff.|   446|
|       Perservere|Didn't they lose ...|Didn't they lose ...|    leagueoflegends|    t5_2rfxx|c6bftvc|Didn't they lose ...|just because you'...|   569|
|      fallsuspect|You probably won'...|You probably won'...|          AskReddit|    t5_

In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='subreddit',outputCol='label')
tokenizer = Tokenizer(inputCol="body", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [0]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [0]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data_df)
cleaned = cleaner.transform(data_df)

In [0]:
indexer = StringIndexer(inputCol="label", outputCol="target") 
indexed = indexer.fit(cleaned).transform(cleaned) 

In [0]:
# Show label and resulting features
indexed.select(['target', 'features']).show()

+------+--------------------+
|target|            features|
+------+--------------------+
|   7.0|(262145,[4547,847...|
|   0.0|(262145,[1696,103...|
|   2.0|(262145,[161,1252...|
|   0.0|(262145,[13781,21...|
|   5.0|(262145,[379,1230...|
|   0.0|(262145,[1619,392...|
|   0.0|(262145,[991,1619...|
|   0.0|(262145,[1546,557...|
|   0.0|(262145,[991,2325...|
|   0.0|(262145,[991,2410...|
|   0.0|(262145,[920,1411...|
|   0.0|(262145,[3772,392...|
|   0.0|(262145,[1260,218...|
|   7.0|(262145,[107,3282...|
|   4.0|(262145,[3564,380...|
|   4.0|(262145,[2050,270...|
|   8.0|(262145,[277,5088...|
|   0.0|(262145,[4978,105...|
|   0.0|(262145,[5451,649...|
|   0.0|(262145,[956,1240...|
+------+--------------------+
only showing top 20 rows



In [0]:
final_df = indexed.select(['features', 'label'])
final_df = final_df.withColumnRenamed('target', 'label')

In [0]:
from pyspark.sql.types import IntegerType
final_df = final_df.withColumn("label", final_df["label"].cast(IntegerType()))

In [0]:
final_df.dtypes

Out[70]: [('features', 'vector'), ('label', 'int')]

In [0]:
final_df.count()

Out[64]: 1365644

In [0]:
final_df1 = final_df.dropna()

In [0]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = final_df1.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-2542230821088227> in <cell line: 7>()
      5 # Create a Naive Bayes model and fit training data
      6 nb = NaiveBayes()
----> 7 predictor = nb.fit(training)

/databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py in patched_method(self, *args, **kwargs)
     28             call_succeeded = False
     29             try:
---> 30                 result = original_method(self, *args, **kwargs)
     31                 call_succeeded = True
     32                 return result

/databricks/spark/python/pyspark/ml/base.py in fit(self, dataset, params)
    203                 return self.copy(params)._fit(dataset)
    204             else:
--> 205                 return self._fit(dataset)
    206         else:
    207             raise TypeError(

/databricks/spark/python/pyspark/ml/wrapper.py in _fit(self, data

In [0]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(5)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-2542230821088238> in <cell line: 2>()
      1 # Tranform the model with the testing data
----> 2 test_results = predictor.transform(testing)
      3 test_results.show(5)

NameError: name 'predictor' is not defined

In [0]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting subreddits was: %f" % acc)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-2542230821088228> in <cell line: 5>()
      3 
      4 acc_eval = MulticlassClassificationEvaluator()
----> 5 acc = acc_eval.evaluate(test_results)
      6 print("Accuracy of model at predicting subreddits was: %f" % acc)

NameError: name 'test_results' is not defined